In [3]:
from langchain.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings,ChatOpenAI
from langchain import hub
import bs4

In [2]:
path_url ="https://docs.python.org/zh-cn/3.11/whatsnew/3.11.html"
loader = WebBaseLoader(web_path=path_url,)
docs = loader.load()

In [4]:
chunk_size = 1000
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap = chunk_overlap)

split_docs = text_splitter.split_documents(docs)

print(f"split_docs[0]:{split_docs[0].page_content}")

split_docs[0]:Python 3.11 有什么新变化 — Python 3.11.8 文档

















































    Theme
    
Auto
Light
Dark



目录

Python 3.11 有什么新变化
摘要 -- 发布重点
新的特性
PEP 657：回溯信息中标注更详细的错误位置
PEP 654：异常组与 except*
PEP 678：可用注释丰富异常
Windows 下的 py.exe 启动器改进


有关类型提示的新增特性
PEP 646：可变参数泛型
PEP 655：将单个 TypedDict 项标记为必填或非必填项
PEP 673：Self 类型
PEP 675：任意字面值字符串类型
PEP 681：数据类变换
未来版本可能不再实现 PEP 563


其他语言特性修改
其他 CPython 实现的改变
新增模块
改进的模块
asyncio
contextlib
dataclasses
datetime
enum
fcntl
fractions
functools
hashlib
IDLE 与 idlelib
inspect
locale
logging
math
operator
os
pathlib
re
shutil
socket
sqlite3
string
sys
sysconfig
tempfile
threading
time
tkinter
回溯
typing
unicodedata
unittest
venv
warnings
zipfile


性能优化
更快的 CPython
更快的启动
冻结导入 / 静态代码对象


更快的运行时
开销更低、更为惰性的 Python 帧
内联的 Python 函数调用
PEP 659：专门化自适应解释器


杂项
常见问题
我要如何编写代码以便应用这些加速？
CPython 3.11 会使用更多内存吗？
我没有发现我的运行负载有任何加速。 为什么？
是否有 JIT 编译器？


关于


CPython 字节码的改变
新的操作码
被替换的操作码
修改/移除的操作码


弃用
语言/内置对象
模块
标准库


In [5]:
# 存入向量数据库
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=split_docs,embedding=OpenAIEmbeddings())
print(f"vectorstore的类型:\n{type(vectorstore)}")


# 检索相关文档
retriever = vectorstore.as_retriever(search_type='similarity',search_kwargs ={"k":5})

vectorstore的类型:
<class 'langchain_chroma.vectorstores.Chroma'>


In [6]:
# 进行检索获取top k
retrieved_docs = retriever.invoke("新增了那些功能")

In [ ]:
print(f"top_0的内容: \n{retrieved_docs[0].page_content} ")

In [7]:
# 大模型进行总结
llm = ChatOpenAI(model="gpt-3.5-turbo")
# 拉取提示词
prompt = hub.pull("rlm/rag-prompt-mistral")
demo_prompt = hub.pull("rlm/rag-prompt")
print(f"prompt:\n {prompt}")

D:\tools\miniconda3\envs\llm-3.11\Lib\site-packages\langsmith\client.py:5499: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


prompt:
 input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt-mistral', 'lc_hub_commit_hash': '5065c9030dae8e9b27fa0dbf6d9fa3a516d26978adc580c95b2ed1541dc366fa'} template="<s> [INST] You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. [/INST] </s> \n[INST] Question: {question} \nContext: {context} \nAnswer: [/INST]"


In [10]:
# LCEL 使用


question_1 = "新增了那些特性和模块"
question_2 = "TaskGroup的使用场景"
question_3 = "timeout能不能在上一个版本使用"
def ask_question(question,prompt=demo_prompt):
    rag_chain = ({"context":retriever,"question":RunnablePassthrough()}
            |prompt|llm|StrOutputParser()
            )
    message = rag_chain.invoke(question)
    print(f" question:{question}\n answer:{message}")

In [12]:
ask_question(question_1)
ask_question(question_2)
ask_question(question_3)


question:新增了那些特性和模块
 answer:新增的模块包括tomllib和wsgiref.types。改进的模块包括asyncio，新增了TaskGroup类等功能。其他语言特性修改包括星号解包表达式在for语句中的使用等。
question:TaskGroup的使用场景
 answer:TaskGroup类是一个异步上下文管理器，可以持有一组任务，等待它们全部完成后才退出。建议在新代码中使用TaskGroup类，而不是直接使用create_task()和gather()。Task类还添加了cancelling()和uncancel()方法，供内部使用，特别是TaskGroup。
question:timeout能不能在上一个版本使用
 answer:我不知道。


In [13]:
ask_question(question_1,prompt=prompt)
ask_question(question_2,prompt=prompt)
ask_question(question_3,prompt=prompt)

question:新增了那些特性和模块
 answer:新增的模块包括tomllib和wsgiref.types。改进的模块包括asyncio中添加了TaskGroup类和其他功能。其他语言特性修改包括新增了一些PEP，如PEP 646、PEP 655等。
question:TaskGroup的使用场景
 answer:TaskGroup类是一个异步上下文管理器，可以持有一组任务，等待它们全部完成后才退出。建议对新代码使用TaskGroup类，而不是直接使用create_task()和gather()。TaskGroup主要供内部使用，特别是为Task添加了cancelling()和uncancel()方法。
question:timeout能不能在上一个版本使用
 answer:I don't know the answer to whether `timeout` can be used in the previous version.


In [ ]:
# 多链路

In [46]:
from operator import itemgetter

model="gpt-3.5-turbo"

def get_prompt(topic):
    return ChatPromptTemplate.from_template(topic)
# 1.创建主题生成器
topic = (
    get_prompt("生成关于{base_topic}的论点")|
    ChatOpenAI(model=model)|
    StrOutputParser()|
    {"base_topic":RunnablePassthrough()}
)
# 2创建链路分支1
chain_for = (
    get_prompt("生成关于{base_topic}的正面理论依据")|
    ChatOpenAI(model=model)|
    StrOutputParser()
   
)
# 3创建链路分支2
chain_against = (
        get_prompt("生成关于{base_topic}的反面理论依据")|
        ChatOpenAI(model=model)|
        StrOutputParser()
      
    )
prompt =ChatPromptTemplate.from_messages(
   [
            ("ai", "{base_topic}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
summary = (
    prompt|
    ChatOpenAI(model=model)|
    StrOutputParser()

)


chain = (topic|
         {  "base_topic": itemgetter("base_topic"),
            "results_1": chain_for,
            "results_2": chain_against,
           
        }|
         summary)

In [49]:
text = chain.invoke({"base_topic": "中国足球"}) # 注意键要和模板定义保持一致

In [50]:
from IPython.display import HTML
HTML(text)